# To expand the grouped table descriptions to a DB version

In [1]:
import numpy as np
import pandas as pd

In [46]:
df_flux = pd.read_csv("logicOrder_dr1_flux_columns.csv")
df_mag = pd.read_csv("logicOrder_dr1_mag_columns.csv")
df_tile = pd.read_csv("logicOrder_dr1_tile_columns.csv")
df_main = pd.read_csv("logicOrder_dr1_main_columns.csv")

## Expand on a case-by-case (band, nside, aperture)

In [48]:
PHOT_APERTURES = [1.85, 3.70, 5.55, 7.41, 11.11, 14.81, 18.52, 22.22, 25.93, 29.63, 44.44, 66.67]

In [49]:
def extend(df_tmp, out_tmp):
    nm, txt = [], []
    aux_b = "{G,R,I,Z,Y}"
    aux_n = "{32,64,1024,4096,16384}"
    aux_a = "{1-12}"
    for index, row in df_tmp.iterrows():
        # Band, no aperture
        if (row["Column Name"].find(aux_b) >= 0) and (row["Column Name"].find(aux_a) < 0):
            list_band = ["g", "r", "i", "z", "Y"]
            for b in list_band:
                s1 = row["One-line description"][0].lower()
                txt0 = list(row["One-line description"])
                txt0[0] = s1
                txt0 = "".join(txt0)
                aux_txt = "{0}-band ".format(b) + txt0
                aux_nm = row["Column Name"].replace(aux_b, b.upper())
                nm.append(aux_nm)
                txt.append(aux_txt)
        # Band and aperture
        elif (row["Column Name"].find(aux_b) >= 0) and (row["Column Name"].find(aux_a) >= 0):
            list_ap = np.arange(1, 13)
            list_band = ["g", "r", "i", "z", "Y"]
            for b in list_band:
                for idx, ap in enumerate(list_ap):
                    s1 = row["One-line description"][0].lower()
                    txt0 = list(row["One-line description"])
                    txt0[0] = s1
                    txt0 = "".join(txt0)
                    aux_txt = "Aperture {1} equivalent to {2} [pixel], {0}-band ".format(b, ap, PHOT_APERTURES[idx]) 
                    aux_txt += txt0
                    aux_nm = row["Column Name"].replace(aux_b, b.upper())
                    aux_nm = aux_nm.replace(aux_a, str(ap))
                    nm.append(aux_nm)
                    txt.append(aux_txt)
        # Hpix
        elif (row["Column Name"].find(aux_n) >= 0):
            list_n = aux_n.replace("{", "").replace("}", "").split(",")
            for n in list_n:
                aux_txt = row["One-line description"]
                aux_txt = aux_txt.replace("its NSIDE", "NSIDE={0}".format(n))
                aux_nm = row["Column Name"]
                aux_nm = aux_nm.replace(aux_n, n)
                nm.append(aux_nm)
                txt.append(aux_txt)
        else:
            nm.append(row["Column Name"])
            txt.append(row["One-line description"])
    # Create a DF with outputs
    ext = pd.DataFrame({"Column Name":nm, "Description":txt})
    ext.to_csv(out_tmp, index=False, header=True)
    return True

In [50]:
extend(df_flux, "extended_dr1_flux_columns.csv")

True

In [51]:
extend(df_mag, "extended_dr1_mag_columns.csv")

True

In [52]:
extend(df_tile, "extended_dr1_tile_columns.csv")

True

In [53]:
extend(df_main, "extended_dr1_main_columns.csv")

True